In [2]:
import pandas as pd
import time
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import warnings, sys, os, gc

import surprise as sp
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV, RandomizedSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, SVD, NMF


## Carga Parcial

In [92]:
df_libros = pd.read_csv('./data/libros.csv')
df_usr = pd.read_csv('./data/usuarios.csv')
df_train = pd.read_csv('./data/opiniones_train.csv')
df_test = pd.read_csv('./data/opiniones_test.csv')

## Regresion Lineal

In [46]:
df_libros.head()

,anio,autor,editorial,genero,idioma,isbn,libro,resumen,titulo
0,2007,"Mutis, Álvaro",DEBOLSILLO,Ficción literaria,Español,9788483464045,empresas-y-tribulaciones-de-magroll-el-gaviero,Reúne en dos volúmenes las siete novelas que c...,EMPRESAS Y TRIBULACIONES DE MAGROLL EL GAVIERO
1,2016,"Ruiz García, Daniel",TUSQUETS,Narrativa,Español,9788490663325,la-gran-ola,Julián Márquez está al límite. Y no sólo por s...,LA GRAN OLA
2,2015,"Spanbauer, Tom",RANDOM HOUSE,Literatura contemporánea,Español,9788439730378,yo-te-quise-mas,Ben fue un iluso al creer que podría amar a un...,YO TE QUISE MÁS
3,2012,"Day, Sylvia",ESPASA,"Romántica, erótica",Español,9788467009651,no-te-escondo-nada,«Gideon Cross apareció en mi vida como un rayo...,NO TE ESCONDO NADA
4,2014,"Scott Fitzgerald, Francis",ALIANZA,Literatura contemporánea,Español,9788420683614,a-este-lado-del-paraiso,La figura de Francis Scott Fitzgerald (1890-19...,A ESTE LADO DEL PARAÍSO


In [115]:
df_libros.loc[0]

anio                                                      2007
autor                                            Mutis, Álvaro
editorial                                           DEBOLSILLO
genero                                       Ficción literaria
idioma                                                 Español
isbn                                             9788483464045
libro           empresas-y-tribulaciones-de-magroll-el-gaviero
resumen      Reúne en dos volúmenes las siete novelas que c...
titulo          EMPRESAS Y TRIBULACIONES DE MAGROLL EL GAVIERO
Name: 0, dtype: object

In [59]:
isbn = df_libros.loc[df_libros['libro'] == 'el-maestro-de-esgrima']['isbn'].values[0]
isbn
#df_ratings_usuario = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")


'9788420472485'

In [74]:
df_libros['libro'][1]

'la-gran-ola'

In [96]:
#prueba['libro'] = prueba['libro'].map(df_libros.set_index('libro')['isbn'])
prueba

,libro,usuario,puntuacion
0,9788420472485,216,8.0
1,9788498002782,288,3.0
2,9788423348138,300,7.0
3,9788490328729,120,10.0
4,9788401336430,1122,5.0
...,...,...,...
42315,9788427032118,1079,8.0
42316,9788467053418,151,8.0
42317,9788439710417,169,7.0
42318,9788433923981,216,6.0


In [108]:
prueba.dropna
prueba.isna().sum()

libro         20
usuario        0
puntuacion     0
dtype: int64

In [104]:
df_libros.isna().sum()


anio           4
autor          4
editorial      9
genero         9
idioma       106
isbn           6
libro          0
resumen      364
titulo         4
dtype: int64

In [15]:
df_train[['libro', 'usuario']]

,libro,usuario
0,el-maestro-de-esgrima,216
1,el-angel-mas-tonto-del-mundo,288
2,un-millon-de-gotas,300
3,el-resplandor,120
4,el-fuego,1122
...,...,...
42315,el-callejon-de-los-milagros,1079
42316,ponte-en-mi-piel,151
42317,veinte-anos-despues,169
42318,la-herencia-de-wilt,216


In [128]:
nan_value = float("NaN")
prueba.replace("", nan_value, inplace=True)
prueba.dropna(subset = ["libro"], inplace=True)

In [132]:
prueba = pd.read_csv('./submission/prueba.csv')

In [133]:
full_x = prueba[['libro', 'usuario']]
full_y = prueba[['puntuacion']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)
print("train: ",len(X_train),len(y_train))
print("test:  ",len(X_test),len(y_test))
# Creo regresion lineal
model_avg = LinearRegression()
model_abc = XGBRegressor()
# Entreno el modelo usando el conjunto de TRAIN
model_avg.fit(X_train, y_train)
model_abc.fit(X_train, y_train)

# Hago predicciones usando el TEST 
prediction_avg = model_avg.predict(X_test)
prediction_abc = model_abc.predict(X_test)
#ECM
print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_abc))
print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_avg))
#train:  29624 29624
#test:   12696 12696
#Mean squared error: 3.0050

train:  29333 29333
test:   12572 12572
Mean squared error: 2.7220
Mean squared error: 3.0153


### Surprise

In [3]:
#df_libros = pd.read_csv('./data/libros.csv')
df_train = pd.read_csv('./data/opiniones_train.csv')
df_test = pd.read_csv('./data/opiniones_test.csv')

In [65]:
#trainset, testset = train_test_split(df_train, test_size=0.30)
trainset = df_train
testset = df_test
print(trainset.shape, testset.shape)

(42320, 3) (10584, 4)


In [66]:
reader = sp.reader.Reader(rating_scale=(1, 10))

data= sp.dataset.Dataset.load_from_df(trainset[['libro', 'usuario', 'puntuacion']], reader)

In [67]:
# opciones del algoritmo
sim_options = {
    'name': 'cosine', 
    'user_based': True, 
    'min_support': 1,
}



#KNNBasic
#RMSE = {'min_k': 6, 'k': 50, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}



# opciones del algoritmo
param_grid_svd = {'n_factors': [80, 90, 100, 110, 120], 'n_epochs': [10, 15, 20, 25, 30], 'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.01, 0.02, 0.03]}


KNN_algo = sp.prediction_algorithms.knns.KNNBasic(k=50, min_k=6, sim_options=sim_options)

SVD_algo = SVD(n_factors= 120, n_epochs= 10, lr_all= 0.008, reg_all= 0.03)
# RMSE = {'n_factors': 90, 'n_epochs': 30, 'lr_all': 0.003, 'reg_all': 0.03}
# RMSE = {'n_factors': 120, 'n_epochs': 10, 'lr_all': 0.008, 'reg_all': 0.03}

In [68]:
trainset_model = data.build_full_trainset()

In [218]:
t_0 = time.time()
svd_rs = RandomizedSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'],n_iter=20, n_jobs=-1)
svd_rs.fit(data)
print("SVD")
print('RMSE =', svd_rs.best_params['rmse']) 
print ("tiempo: ", time.time() - t_0)

SVD
RMSE = {'n_factors': 120, 'n_epochs': 10, 'lr_all': 0.008, 'reg_all': 0.03}
tiempo:  92.61084079742432


In [69]:
t_0 = time.time()
pred_SVD = SVD_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)

tiempo:  1.9440243244171143


In [41]:
t_0 = time.time()
pred_KNN = KNN_algo.fit(trainset_model)
print ("tiempo: ", time.time() - t_0)

Computing the cosine similarity matrix...
Done computing similarity matrix.
tiempo:  21.34980797767639


In [70]:
print(len(testset))

10584


In [71]:
testset.head()

,id,libro,usuario,puntuacion
0,1,los-hijos,201,NaN
1,2,el-temor-de-un-hombre-sabio,299,NaN
2,3,leon-bocanegra,126,NaN
3,4,el-caballero-errante-2,107,NaN
4,5,el-mar-el-mar,85,NaN


In [72]:
def full_prediction(algo_name,algoritmo,test):
    list_pred = []
    for index, row in tqdm_notebook(test.iterrows()):
        row_pred = dict()
        uid = row.libro
        iid = row.usuario
        pred = algoritmo.predict(uid, iid, verbose=False)
        row_pred["id"] = row.id
        #row_pred["id"] = row.libro
        row_pred["puntuacion"] = np.around(pred.est, 2)
        list_pred.append(row_pred)
    #print(row.ID.astype(int))
    df = pd.DataFrame(list_pred)    
    
    #print(algo_name + " MSE:      %.4f" % np.sqrt(mean_squared_error(test.puntuacion, df.puntuacion)))
    return list_pred

In [73]:
df_list_pred_SVD = pd.DataFrame(full_prediction("SVD",SVD_algo,testset))
#df_list_pred_KNN = pd.DataFrame(full_prediction("KNN",KNN_algo,testset))

## KNN

In [74]:
df_list_pred_SVD

,id,puntuacion
0,1,7.11
1,2,9.08
2,3,7.81
3,4,7.68
4,5,6.63
...,...,...
10579,10580,8.13
10580,10581,6.32
10581,10582,7.27
10582,10583,9.25


In [75]:
df_list_pred_KNN.to_csv("./submission/ignacio_submit_SVD.csv", index=False)